In [1]:
import tensorflow as tf

import numpy as np

import pylab as pl
%pylab inline

import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys

sys.path.append(r"../Commons")
import ptf_02 as ptf

In [3]:
import itertools as itt

In [4]:
## changing the width of cells
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [5]:
def get_game_passage_probas_in_one_set(gps, first_server = None):
    """
    gps = game proba server
    gps is indexed by (pl, i, j) in {0,1}x{0,..6}x{0,..6}
    Here pl is the player serving, (i,j) is the current state 
    i = games won by pl, j = games won by the non-server
    """
    
    ##########
    ## First deal with cases when first_server != 0
    
    if first_server is None:
        return 1/2 * (
            get_game_passage_probas_in_one_set(gps, 0) + get_game_passage_probas_in_one_set(gps, 1) 
        )

    if first_server == 1:
        return get_game_passage_probas_in_one_set([gps[1], gps[0]], 0).T
    
    #########
    ## So in the rest we can assume that the first server is 0
    
    ## active nodes
    active = list(itt.product(range(6), range(6))) + [(6, 5), (5, 6), (6, 6)]
    
    def gp0(i,j):
        """Returns proba of the 0-th player to win this game."""
        if (i + j) % 2 == 0:
            return gps[0][i,j]
        else:
            return 1 - gps[1][j,i]

    if isinstance(gps[0], np.ndarray):
        dtype = gps[0].dtype
    else:
        dtype = np.object
        
    p = np.zeros((8, 8)).astype(dtype)
    p[0, 0] = 1
    for s in range(1, 15):
        for i in range(s + 1):
            j = s - i
            
            if (i-1, j) in active:
                p[i,j] = p[i,j] + p[i-1, j] * gp0(i-1, j)
            
            if (i, j-1) in active:
                p[i,j] = p[i,j] + p[i, j-1] * (1 - gp0(i, j-1))
                
                
    return p
    

In [6]:
probas = get_game_passage_probas_in_one_set(
    gps = [0.7 * np.ones((7,7)), 0.7 * np.ones((7,7))],
    first_server = None,
)
probas.round(3)

array([[ 1.   ,  0.5  ,  0.21 ,  0.105,  0.044,  0.022,  0.009,  0.   ],
       [ 0.5  ,  0.58 ,  0.395,  0.244,  0.144,  0.077,  0.038,  0.   ],
       [ 0.21 ,  0.395,  0.425,  0.334,  0.24 ,  0.158,  0.073,  0.   ],
       [ 0.105,  0.244,  0.334,  0.349,  0.294,  0.228,  0.114,  0.   ],
       [ 0.044,  0.144,  0.24 ,  0.294,  0.303,  0.266,  0.13 ,  0.   ],
       [ 0.022,  0.077,  0.158,  0.228,  0.266,  0.272,  0.136,  0.057],
       [ 0.009,  0.038,  0.073,  0.114,  0.13 ,  0.136,  0.158,  0.079],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.057,  0.079,  0.   ]])

In [7]:
set_final_states = [(6, i) for i in range(0,5)] + [(i, 6) for i in range(0,5)] + [(7, 5), (7, 6), (5, 7), (6, 7)]
set_final_states

[(6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (0, 6),
 (1, 6),
 (2, 6),
 (3, 6),
 (4, 6),
 (7, 5),
 (7, 6),
 (5, 7),
 (6, 7)]

In [8]:
## win proba for player 0
sum( probas[i, j] for i, j in set_final_states if i > j)

0.5

In [9]:
## under 9.5 proba
sum( probas[i, j] for i, j in set_final_states if i + j < 9.5)

0.46879265999999997

## Try in TF

In [10]:
tf_gps0 = tf.placeholder(dtype = "float", shape = [7, 7], name = "gps0")    
tf_gps1 = tf.placeholder(dtype = "float", shape = [7, 7], name = "gps1")    

In [11]:
tf_probas = get_game_passage_probas_in_one_set([tf_gps0, tf_gps1])

In [12]:
tf_wp0 = sum( tf_probas[i,j] for i, j in set_final_states if i > j )

In [13]:
sess = tf.InteractiveSession() 

In [14]:
sess.run(tf_wp0, 
    feed_dict ={
        tf_gps0: 0.7 * np.ones((7,7)), 
        tf_gps1: 0.7 * np.ones((7,7)), 
    }
)

0.5

### linear model

In [15]:
def get_game_passage_probas_in_one_set_linear_model_tf(le, explanatory, first_server = None): 
    """
    le.shape = (7, 7, 3)
    explanatory.shape = (?, 3)
    """
    switch = np.array(
        [[0, 1, 0], [1, 0, 0], [0, 0, 1]],
        dtype = np.float
    )
    
    gps0 = ptf.logist(tf.einsum("ijk,nk", le, explanatory))
    gps1 = ptf.logist(tf.einsum("ijk,nk", le, tf.matmul(explanatory, switch)))
    return get_game_passage_probas_in_one_set(
        gps = [gps0, gps1],
        first_server = None,
    )


In [16]:
import numpy.random as rnd

In [17]:
le = rnd.random(size = 7 * 7 * 3).reshape([7, 7, 3])

In [18]:
expl = rnd.rand(10, 3)

In [19]:
probas_tf = get_game_passage_probas_in_one_set_linear_model_tf( tf.constant(le), tf.constant(expl))

In [20]:
sess.run(
        sum( probas_tf[i,j] for i, j in set_final_states if i > j )
    )

array([ 0.48393132,  0.53906265,  0.49127853,  0.49953771,  0.5605586 ,
        0.51880966,  0.47393575,  0.56382136,  0.53329533,  0.5211279 ])

In [21]:
sess.run(
        sum( probas_tf[i,j] for i, j in set_final_states if i+j < 9.5 )
    )

array([ 0.47682095,  0.4901628 ,  0.51671739,  0.50802544,  0.49618278,
        0.49228657,  0.46979532,  0.50201041,  0.51613763,  0.50594515])

We will compare these results with ones calculated in the old theano way

In [22]:
import theano as th
import theano.tensor as tt

In [23]:
sys.path.append(r"../Tennis_01")
import tennis_commons_03 as tc

In [24]:
for old_p in tc.get_theano_book_probas(
    tt.constant(le), 
    explanatory = tt.constant(expl)
):
    print(old_p.eval())

[ 0.48393132  0.53906265  0.49127853  0.49953771  0.5605586   0.51880966
  0.47393575  0.56382136  0.53329533  0.5211279 ]
[ 0.47682095  0.4901628   0.51671739  0.50802544  0.49618278  0.49228657
  0.46979532  0.50201041  0.51613763  0.50594515]


## Try in Theano

In [25]:
th_gps0 = tt.matrix(name = "gps0")
th_gps1 = tt.matrix(name = "gps1")

In [26]:
th_probas = get_game_passage_probas_in_one_set([th_gps0, th_gps1])

In [27]:
th_wp0 = sum( th_probas[i,j] for i, j in set_final_states if i > j )

In [28]:
th_wp0.eval({
            th_gps0: 0.7 * np.ones((7,7)), 
            th_gps1: 0.7 * np.ones((7,7)), 
        })

array(0.5)

# Set-evolution

In [29]:
def get_set_passage_probas_in_match(sp0):
    """
    sp0 = probas of 0th player winning the set
    sp0 is indexed by (i, j) in {0,1}x{0,1}
    Here (i,j) is the current state i.e. sets already won by each player
    """
    
    ## active nodes
    active = list(itt.product(range(2), range(2))) 
    
    if isinstance(sp0, np.ndarray):
        dtype = sp0.dtype
    else:
        dtype = np.object
        
    p = np.zeros((3, 3)).astype(dtype)
    p[0, 0] = 1
    for s in range(1, 5):
        for i in range(s + 1):
            j = s - i
            
            if (i-1, j) in active:
                p[i,j] = p[i,j] + p[i-1, j] * sp0[i-1, j]
            
            if (i, j-1) in active:
                p[i,j] = p[i,j] + p[i, j-1] * (1 - sp0[i, j-1])
                
                
    return p
    

In [30]:
match_probas = get_set_passage_probas_in_match(
    sp0 = 0.3 * np.ones((2, 2))
)
match_probas

array([[ 1.   ,  0.7  ,  0.49 ],
       [ 0.3  ,  0.42 ,  0.294],
       [ 0.09 ,  0.126,  0.   ]])

In [31]:
match_probas = get_set_passage_probas_in_match(
    sp0 = 0.7 * np.ones((2, 2))
)
match_probas

array([[ 1.   ,  0.3  ,  0.09 ],
       [ 0.7  ,  0.42 ,  0.126],
       [ 0.49 ,  0.294,  0.   ]])

# Bundle to one function returning all probas we need

In [32]:
match_final_states = [(2, 0), (2, 1), (0, 2), (1, 2) ]

In [33]:
match_all_states = list(itt.product(range(2), range(2)))
match_all_states

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [34]:
def get_game_and_set_passage_probas_linear_model_tf(les, explanatory):
    """
    returns game_pass_probas and set_pass_probas
    game_pass_probas[i,j] are the passage probas of the games in the set (i,j)
    conditional on the event that the set (i,j) is played
    """
    game_pass_probas = np.zeros((3, 3), dtype=np.object)
    for i,j in match_all_states:
        game_pass_probas[i,j] = get_game_passage_probas_in_one_set_linear_model_tf( 
            le =les[i, j],
            explanatory=explanatory
        )

    # sp0 = probabilities of winning the set for the 0th player
    sp0 = np.zeros((2, 2), dtype=np.object)
    for s0, s1 in match_all_states:
        sp0[s0, s1] = sum( game_pass_probas[s0,s1][g0, g1] for g0, g1 in set_final_states if g0 > g1 )

    set_pass_probas = get_set_passage_probas_in_match(sp0)
    
    return game_pass_probas, set_pass_probas
    


In [35]:
les_shape = (2, 2, 7, 7, 3)
les = rnd.rand(*les_shape)

In [36]:
game_pass_probas, set_pass_probas = get_game_and_set_passage_probas_linear_model_tf(
    les = tf.constant(les), explanatory = tf.constant(expl)
)

In [37]:
sp0 = np.zeros((2, 2), dtype=np.object)
for s0, s1 in match_all_states:
    sp0[s0, s1] = sum( game_pass_probas[s0,s1][g0, g1] for g0, g1 in set_final_states if g0 > g1 )
for s0, s1 in match_all_states:
    print(s0, s1, sess.run(sp0[s0, s1]))


0 0 [ 0.49961272  0.50273476  0.50004467  0.49997262  0.50404384  0.50152109
  0.49796404  0.50358699  0.50025447  0.50142594]
0 1 [ 0.49593622  0.50860117  0.49797159  0.49989989  0.51332311  0.5040372
  0.49419768  0.51417957  0.50774636  0.50453315]
1 0 [ 0.49687837  0.50570038  0.49810136  0.4999218   0.50939432  0.50247549
  0.49693695  0.51098559  0.50738634  0.50333028]
1 1 [ 0.50487035  0.48690276  0.5020187   0.50014257  0.48007635  0.49366707
  0.5096822   0.47979389  0.49145122  0.49334107]


In [38]:
# match win proba for the first player
mwp0 = sum( set_pass_probas[s0, s1] for s0,s1 in match_final_states if s0 > s1 )

In [92]:
sess.run(mwp0)

array([ 0.49956045,  0.50118608,  0.49907764,  0.50225292,  0.50052514,
        0.50237877,  0.50094011,  0.49788357,  0.49997028,  0.50037385])

### test

In [94]:
switch = np.array(
        [[0, 1, 0], [1, 0, 0], [0, 0, 1]],
        dtype = np.float
    )

game_pass_probas, set_pass_probas = get_game_and_set_passage_probas_linear_model_tf(
    les = tf.constant(les), explanatory = tf.constant(expl @ switch)
)

mwp0 = sum( [set_pass_probas[s0, s1] for s0,s1 in match_final_states if s0 > s1 ])
mwp1 = sum( [set_pass_probas[s0, s1] for s0,s1 in match_final_states if s0 < s1 ])

print(sess.run(mwp0))
print(sess.run(mwp1))

[ 0.50047066  0.49887465  0.50108319  0.49805621  0.49950228  0.49841304
  0.49919562  0.50288254  0.50002997  0.50058748]
[ 0.49952934  0.50112535  0.49891681  0.50194379  0.50049772  0.50158696
  0.50080438  0.49711746  0.49997003  0.49941252]


In [95]:
sp0 = np.zeros((2, 2), dtype=np.object)
for s0, s1 in match_all_states:
    sp0[s0, s1] = sum( game_pass_probas[s0,s1][g0, g1] for g0, g1 in set_final_states if g0 > g1 )
for s0, s1 in match_all_states:
    print(s0, s1, 1- sess.run(sp0[s0, s1]))


0 0 [ 0.49857097  0.50240597  0.49673371  0.50517475  0.50147599  0.50726195
  0.5029624   0.49240718  0.49987719  0.49286606]
0 1 [ 0.50517748  0.49407645  0.51141327  0.48635788  0.49581855  0.47565945
  0.48929405  0.52260709  0.50050156  0.52872403]
1 0 [ 0.49342371  0.50904014  0.48531609  0.51981523  0.50581011  0.53155464
  0.51354125  0.46912084  0.49941044  0.46781786]
1 1 [ 0.50121823  0.49834713  0.50289612  0.49593515  0.49873254  0.49309601
  0.4973644   0.50673074  0.50010712  0.50864979]


In [84]:
[(s0, s1) for s0,s1 in match_final_states if s0 > s1 ]

[(2, 0), (2, 1)]

# Bordel

## Comparting with previous way

In [ ]:
from tennis_commons_03 import logist, log_odds

In [ ]:
p0, p1 = 0.7, 0.8 
expl0 = np.array([log_odds(p0), log_odds(p1), 1])
expl1 = np.array([log_odds(p1), log_odds(p0), 1])

In [ ]:
for old_p in tc.get_theano_book_probas(
    tt.constant(le), 
    explanatory = tt.constant(expl0[None, :])
):
    print(old_p.eval())

New way

In [ ]:
probas = get_game_passage_probas_in_one_set(
    gps = [logist(le @ expl0), logist(le @ expl1)],
    first_server = None,
)

In [ ]:
## win proba for player 0
sum( probas[i, j] for i, j in set_final_states if i > j)

In [ ]:
## under 9.5 proba
sum( probas[i, j] for i, j in set_final_states if i + j < 9.5)

We can use it to get a new version of `tc.get_theano_book_probas`

In [ ]:
def get_theano_book_probas(le_th, explanatory):
    switch = np.array(
        [[0, 1, 0], [1, 0, 0], [0, 0, 1]]
    )
    expl0 = explanatory.T
    expl1 = tt.dot(switch, expl0)
    
    gps0 = logist(tt.dot(le, expl0))
    gps1 = logist(tt.dot(le, expl1))
    probas = get_game_passage_probas_in_one_set(
        gps = [gps0, gps1],
        first_server = None,
    )
    
    ## win proba for player 0
    p_ha = sum( probas[i, j] for i, j in set_final_states if i > j)
    ## under 9.5 proba
    p_ou = sum( probas[i, j] for i, j in set_final_states if i + j < 9.5)
    
    return p_ha, p_ou

In [ ]:
for th_p in get_theano_book_probas(
    tt.constant(le), 
    explanatory = tt.constant(expl0[None, :])
):
    print(th_p.eval())

Stack

## Visualization

In [ ]:
with tf.Session() as session:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("tmp/basic", session.graph)

Now you 
* run the command-prompt for this winpython. 
* in the command prompt put `tensorboard --logdir=`path/to/log-directory
* in a browser open `localhost:6006`